# Plotting trait-fMRI correlations

(older filename: 2A_trait_fMRI_correl_analyses_nooutputs_Figure7d.ipynb)
 Rekha Varrier, 2022

In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams.update({'font.size': 16})

In [ ]:
# setting default fontsizes for plots

s=20 # CHANGE FONTSIZE HERE

plt.rc('font', size=s) #controls default text size
plt.rc('axes', titlesize=s) #fontsize of the title
plt.rc('axes', labelsize=s) #fontsize of the x and y labels
plt.rc('xtick', labelsize=s) #fontsize of the x tick labels
plt.rc('ytick', labelsize=s) #fontsize of the y tick labels
plt.rc('legend', fontsize=s) #fontsize of the legend
#import matplotlib as mpl
#mpl.rcParams['font.weight']= 'normal'


In [ ]:
res_behav_data = pd.read_csv('../data/RESTRICTED_esfinn_11_21_2021_19_19_35.csv')
res_behav_data.set_index("Subject", inplace=True)
res_behav_data.index = res_behav_data.index.map(str)
print(res_behav_data.shape)
res_behav_data.head()

Read in unrestricted behavioral data:

In [ ]:
unres_behav_data = pd.read_csv('../data/unrestricted_esfinn_11_21_2021_19_19_13.csv')
unres_behav_data.set_index("Subject", inplace=True)
unres_behav_data.index = unres_behav_data.index.map(str)
print(unres_behav_data.shape)
unres_behav_data.head()

In [ ]:
behav_data = pd.concat([res_behav_data, unres_behav_data], axis=1)
print(behav_data.shape)
behav_data.head()

Read in FMRI data, join into single dataframe, then join this with the larger dataframe:

In [ ]:
#suff= ''
suff = '_corrected'

In [ ]:
data_file_loc = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/data'
sub_id_all = np.load(os.path.join(data_file_loc,f'sub_ID_all{suff}.npy'))
sub_id_all = [str(i) for i in sub_id_all]
print(len(sub_id_all),sub_id_all[:5])

# load behavioral responses too - to identify missed/invalid trials later - 1 for "social", 0 for "nonsocial" and 9 for "unsure", nan for missed response
responses = np.load(os.path.join(data_file_loc,f'responses{suff}.npy'))
responses.shape # subs *movies

In [ ]:
x = np.load('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/data/sub_ID_all_corrected.npy')
x,x.shape, responses.shape # if not 1049,10 - check that the right file is imported

In [ ]:
shen268_lbl = pd.read_csv(os.path.join(data_file_loc,"shen_dictionary.csv"))
pd.set_option('display.max_colwidth', None)
shen268_lbl.head()

In [ ]:
# for comparison with the other main figure if needed
nodes_coaxbill_rand_all = np.load(os.path.join(data_file_loc,'nodes_coaxbill_rand_all.npy')) # sig nodes hihglighted in the first S>NS GLM
len(np.where(nodes_coaxbill_rand_all)[0])

In [ ]:
from multipy.fdr import lsu
from corrstats import independent_corr
vidnames = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical","Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]


# Traits vs. fmri unmodulated by responses

In [ ]:
from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain
from nilearn.plotting import plot_glass_brain, plot_surf_roi,plot_stat_map,plot_img,plot_surf_contours
from nilearn import datasets
from nilearn.surface import vol_to_surf

import nibabel as nib

from nilearn.datasets import fetch_surf_fsaverage
fsaverage = fetch_surf_fsaverage()

#coords = [10*int(i) for i in np.linspace(-4,7,6)]
bg_img = datasets.load_mni152_template()

mask = Brain_Data('https://neurovault.org/media/images/8423/shen_2mm_268_parcellation.nii.gz')
mask_x = expand_mask(mask)


def color_rois(values):
    
    """
    This function assumes you are passing a vector "values" with the same length as the number of nodes in the atlas.
    """
    shen268 = nib.load(os.path.join(data_file_loc,"shen_2mm_268_parcellation.nii.gz"))
    shen268_data = shen268.get_fdata()
    img = np.zeros(shen268_data.shape)
    #print(shen268_data.shape)
    for roi in range(len(values)):
        itemindex = np.where(shen268_data==roi+1) # find voxels in this node (add 1 to account for zero-indexing)
        #print(len(itemindex[0]))
        img[itemindex] = values[roi] # color them by the desired value 

    affine = shen268.affine
    img_nii = nib.Nifti1Image(img, affine)
    
    return img_nii

fig_save_loc = os.path.join('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/paper_prep/figures/fig2_2_traits/fmri_results')

In [ ]:
def explore_correlation(x, y, data,corrtype='Spearman'):
    """
    Calculates and plots correlation between x and y variables in dataframe `data`, plus distribution of x and y 
    """
    sns.set_style("white")
    
    inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) # find rows where neither x or y is NaN

    g = sns.jointplot(x=x, y=y, data=data, kind='reg', color='gray')

    # Calculate and print correlations
    if corrtype == 'Pearson':
        rp, pp = stats.pearsonr(data[x][inds], data[y][inds])
        g.ax_joint.annotate(f'r_p = {rp:.2f} (p={pp:.1g})', xy=(.05,.95), xycoords='axes fraction')
    elif corrtype == 'Spearman':
        rs, ps = stats.spearmanr(data[x][inds], data[y][inds])
        g.ax_joint.annotate(f'r_s = {rs:.2f} (p={ps:.1g})', xy=(.05,.95), xycoords='axes fraction')
    
    return g

# ALL MOVIES

In [ ]:
# first of all, best to remove subjects with < 10 responses for power in general
count_resp = np.zeros((responses.shape[0],))
for i in range(responses.shape[0]):
    count_resp[i] = len(np.where(~np.isnan(responses[i,:]))[0])
subs_10resp = np.where(count_resp == 10)[0]
len(subs_10resp)

In [ ]:
# creating a 3D array of beta coeffs across movies from individual movie files
all_coefs_run = np.zeros((1049,268,10))
#all_coefs_trial = np.zeros((1049,268,10))

vidnames = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical","Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]
for m in range(10):
    fileName =  os.path.join(data_file_loc,'coefs_run_norm','slope_reg',f'coef_slopereg_runnorm_{vidnames[m]}{suff}.npy')
    #fileName =  os.path.join(data_file_loc,'coefs_trial_norm','slope_reg',f'coef_slopereg_trialnorm_{vidnames[m]}{suff}.npy')
    all_coefs_run[:,:,m] = np.load(fileName)

In [ ]:
from pymer4.models import Lmer # ,Lm # for LME
import time

In [ ]:
data = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
n=0
nresp = [len(np.where(~np.isnan(responses[i,:]))[0]) for i in range(responses.shape[0])]
for m in range(10):
    #fMRI_data = pd.DataFrame({'coefs': all_coefs[subs_10resp,n,m],'response':responses[subs_10resp,m], 'subID':sub_id_all[subs_10resp], 'movie':np.repeat(m,len(subs_10resp))})
    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node': all_coefs_run[:,n,m], 'response': responses[:,m],
    'nresp':nresp, 'movie': np.repeat(m,1049)})
    #fMRI_data = fMRI_data.loc[fMRI_data['nresp']==10,:]
    fMRI_data.set_index("Subject", inplace=True)
    data_temp = fMRI_data.join(behav_data.loc[:,['ASR_Intn_T','Age_in_Yrs','Gender']], how='inner') # join betas and trait info
    data = data.append(data_temp)#,ignore_index=True)
data.reset_index(inplace=True)

inds = ~np.isnan(data['slopeReg_node']) # find rows with missing fMRI data
data = data.loc[inds,:]
data.shape

# Demographics for the paper

In [ ]:
unique_subs = data.drop_duplicates(subset=['Subject'])
unique_subs

In [ ]:
unique_subs.groupby(['Gender']).count(),unique_subs['Age_in_Yrs'].describe()

# ALL MOVIES - LME - run norm slope betas

In [ ]:
# ALL MOVIES - LME - run norm slope betas
coef_nodes_all_lme_run,   pval_nodes_all_lme_run   =  [np.empty((268,)) for _ in range(2)] # nodes * responses (non-social, social) - LME
coef_nodes_all_lme_run[:],pval_nodes_all_lme_run[:] = [np.nan] *2

nsubs_node_all = []
nresp = [len(np.where(~np.isnan(responses[i,:]))[0]) for i in range(responses.shape[0])]
start_time = time.time()
for n in range(268): 
    if n% 50 ==0:
        print('node',n, ', time taken=',time.time()-start_time)

    data = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
    for m in range(10):
        fMRI_data = pd.DataFrame({'Subject': sub_id_all,'slopeReg_node': all_coefs_run[:,n,m], 
        'response': responses[:,m],'nresp':nresp, 'movie': np.repeat(m,1049)})
        fMRI_data.set_index("Subject", inplace=True)
        data_temp = fMRI_data.join(behav_data['ASR_Intn_T'], how='inner') # join betas and trait info
        data = data.append(data_temp)#,ignore_index=True)
    data.reset_index(inplace=True)

    inds = ~np.isnan(data['slopeReg_node']) & ~np.isnan(data['ASR_Intn_T']) # find rows where neither x or y is NaN
    data = data.loc[inds,:]
    nsubs_node_all.append(len(np.where(np.unique(data['Subject']))[0]))
    # Calculate and print correlations
    
    model2 = Lmer('slopeReg_node ~  ASR_Intn_T + (1|movie)', data=data) # ff: mean response, rf:subjID
    model2.fit(summary = False, verbose = False, no_warnings = True)

    if (len(model2.warnings) == 0):
        coef_nodes_all_lme_run[n] = model2.coefs['Estimate'][1]
        pval_nodes_all_lme_run[n] = model2.coefs['P-val'][1]
   
#pval_nodes_fdr_all_lr_run = lsu(pval_nodes_all_lr_run,q=.05)
pval_nodes_all_lme_run_unc = pval_nodes_all_lme_run < .05
pval_nodes_fdr_all_lme_run = lsu(pval_nodes_all_lme_run,q=.05)
np.where(pval_nodes_fdr_all_lme_run)#, np.where(pval_nodes_fdr_all_lr_run)


In [ ]:
# (only subs with all 10 responses) ALL MOVIES - LME - run norm slope betas 
coef_nodes_all_lme_run_10resp,   pval_nodes_all_lme_run_10resp   =  [np.empty((268,)) for _ in range(2)] # nodes * responses (non-social, social) - LME
coef_nodes_all_lme_run_10resp[:],pval_nodes_all_lme_run_10resp[:] = [np.nan] *2

nsubs_node_all10 = []
nresp = [len(np.where(~np.isnan(responses[i,:]))[0]) for i in range(responses.shape[0])]
start_time = time.time()
for n in range(268): 
    if n% 50 ==0:
        print('node',n, ', time taken=',time.time()-start_time)

    data1 = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
    for m in range(10):
        fMRI_data = pd.DataFrame({'Subject':np.array(sub_id_all)[subs_10resp], 'slopeReg_node': all_coefs_run[subs_10resp,n,m],
        'response':responses[subs_10resp,m], 'nresp':np.array(nresp)[subs_10resp], 'movie':np.repeat(m,len(subs_10resp))})
        fMRI_data.set_index("Subject", inplace=True)
        data_temp = fMRI_data.join(behav_data['ASR_Intn_T'], how='inner') # join betas and trait info
        data1 = data1.append(data_temp)#,ignore_index=True)
    data1.reset_index(inplace=True)

    inds = ~np.isnan(data1['slopeReg_node']) & ~np.isnan(data1['ASR_Intn_T']) # find rows where neither x or y is NaN
    data1 = data1.loc[inds,:]
    # Calculate and print correlations
    nsubs_node_all10.append(len(np.where(np.unique(data1['Subject']))[0]))
    
    model2 = Lmer('slopeReg_node ~  ASR_Intn_T + (1|movie)', data=data1) # ff: mean response, rf:subjID
    model2.fit(summary = False, verbose = False, no_warnings = True)

    if (len(model2.warnings) == 0):
        coef_nodes_all_lme_run_10resp[n] = model2.coefs['Estimate'][1]
        pval_nodes_all_lme_run_10resp[n] = model2.coefs['P-val'][1]
   
#pval_nodes_fdr_all_lr_run = lsu(pval_nodes_all_lr_run,q=.05)
pval_nodes_all_lme_run_unc_10resp = pval_nodes_all_lme_run_10resp < .05
pval_nodes_fdr_all_lme_run_10resp = lsu(pval_nodes_all_lme_run_10resp,q=.05)
np.where(pval_nodes_fdr_all_lme_run_10resp)#, np.where(pval_nodes_fdr_all_lr_run)

In [ ]:
# nr of sig. nodes, FDR-corrected and uncorrected
print('nsubs=', data.shape, len(np.where(pval_nodes_fdr_all_lme_run)[0]), len(np.where(pval_nodes_all_lme_run<.05)[0]))

# ALL 10 RESP; nr of sig. nodes, FDR-corrected and uncorrected
print('nsubs=', data1.shape, len(np.where(pval_nodes_fdr_all_lme_run_10resp)[0]), len(np.where(pval_nodes_all_lme_run_10resp<.05)[0]))

In [ ]:
coef_nodes_all_lme_run[pval_nodes_all_lme_run_unc].shape, len(np.where(coef_nodes_all_lme_run[pval_nodes_all_lme_run_unc]>0)[0])\
    , len(np.where(coef_nodes_all_lme_run[pval_nodes_all_lme_run_unc]<0)[0])

# Characterizing correlations

In [ ]:
print('nr of nodes whose reg coeffts (q<.05 are sig):', coef_nodes_all_lme_run[pval_nodes_fdr_all_lme_run].shape)
print('nr of sig (q<.05) nodes whose reg coeffts (i.e., trait-beta_activity) are: ')
print('> 0:',len(np.where(coef_nodes_all_lme_run[pval_nodes_fdr_all_lme_run]>0)[0]))
print('< 0:',len(np.where(coef_nodes_all_lme_run[pval_nodes_fdr_all_lme_run]<0)[0]))

print('only subs with all 10 responses:')
print('nr of nodes whose reg coeffts (q<.05 are sig):', coef_nodes_all_lme_run_10resp[pval_nodes_fdr_all_lme_run_10resp].shape)
print('nr of sig (q<.05) nodes whose reg coeffts (i.e., trait-beta_activity) are: ')
print('> 0:',len(np.where(coef_nodes_all_lme_run_10resp[pval_nodes_fdr_all_lme_run_10resp]>0)[0]))
print('< 0:',len(np.where(coef_nodes_all_lme_run_10resp[pval_nodes_fdr_all_lme_run_10resp]<0)[0]))

In [ ]:
for i in np.where(coef_nodes_all_lme_run[pval_nodes_fdr_all_lme_run]>0)[0]:
    n = np.where(pval_nodes_fdr_all_lme_run)[0][i]
    x = all_coefs_run[:,n,:].flatten()
    print(f'Node_ind {n}: {np.nanmean(x):.2f}')

# How many nodes show activation and how many deactivation here?

## All subs

In [ ]:

df = pd.DataFrame({'Node': [n+1 for n in np.where(pval_nodes_fdr_all_lme_run)[0]], 'reg_coefft':coef_nodes_all_lme_run[pval_nodes_fdr_all_lme_run], 
'slope_beta': [np.nanmean(all_coefs_run[:,n,:].flatten()) for n in np.where(pval_nodes_fdr_all_lme_run)[0]],
'pval':pval_nodes_all_lme_run[pval_nodes_fdr_all_lme_run], 
'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in np.where(pval_nodes_fdr_all_lme_run)[0]]}).sort_values(by=['reg_coefft'],ascending = True)


## only all 10 subs

In [ ]:
df10 = pd.DataFrame({'Node': [n+1 for n in np.where(pval_nodes_fdr_all_lme_run_10resp)[0]], 'reg_coefft':coef_nodes_all_lme_run_10resp[pval_nodes_fdr_all_lme_run_10resp], 
'slope_beta': [np.nanmean(all_coefs_run[subs_10resp,n,:].flatten()) for n in np.where(pval_nodes_fdr_all_lme_run_10resp)[0]],
'pval':pval_nodes_all_lme_run_10resp[pval_nodes_fdr_all_lme_run_10resp], 
'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in np.where(pval_nodes_fdr_all_lme_run_10resp)[0]]}).sort_values(by=['reg_coefft'],ascending = True)
df10.shape

# Intersection with GLM nodes

In [ ]:
print('GLM nodes (q<.05 across conditions; black contours in figs:', np.where(nodes_coaxbill_rand_all))
print('\nall subs:')
print('all nodes showing trait-dependence (q<.05):', np.where(pval_nodes_fdr_all_lme_run))

print('\nonly subs with all 10 responses:')
print('all nodes showing trait-dependence (q<.05):', np.where(pval_nodes_fdr_all_lme_run_10resp))

In [ ]:
GLM_trait_nodes =  np.where(nodes_coaxbill_rand_all& pval_nodes_fdr_all_lme_run)[0]
GLM_trait_nodes_unc = np.where(nodes_coaxbill_rand_all& pval_nodes_all_lme_run_unc)[0]
print('GLM intersection nodes that also show trait-dependence (FDR):', GLM_trait_nodes, len(GLM_trait_nodes))
print('GLM intersection nodes that also show trait-dependence (unc.):', GLM_trait_nodes_unc, len(GLM_trait_nodes_unc))

GLM_trait_nodes_10resp =  np.where(nodes_coaxbill_rand_all& pval_nodes_fdr_all_lme_run_10resp)[0]
GLM_trait_nodes_unc_10resp = np.where(nodes_coaxbill_rand_all& pval_nodes_all_lme_run_unc_10resp)[0]
print('\n\nGLM intersection nodes that also show trait-dependence (FDR, ONLY 10 RESP SUBJECTS):', GLM_trait_nodes_10resp, len(GLM_trait_nodes_10resp))
print('GLM intersection nodes that also show trait-dependence (unc., ONLY 10 RESP SUBJECTS):', GLM_trait_nodes_unc_10resp, len(GLM_trait_nodes_unc_10resp))

In [ ]:
# non-glm nodes showing trait-dependence
NoGLM_trait_nodes =  np.where(~nodes_coaxbill_rand_all& pval_nodes_fdr_all_lme_run)[0]
print('% of GLM nodes showing trait-dependence:', 100*len(GLM_trait_nodes)/(len(GLM_trait_nodes) + len(NoGLM_trait_nodes)))

NoGLM_trait_nodes_10resp =  np.where(~nodes_coaxbill_rand_all& pval_nodes_fdr_all_lme_run_10resp)[0]
print('% of GLM nodes showing trait-dependence:', 100*len(GLM_trait_nodes_10resp)/(len(GLM_trait_nodes_10resp) + len(NoGLM_trait_nodes_10resp)))

In [ ]:
pd.DataFrame({'Node': [n+1 for n in GLM_trait_nodes_10resp], 'reg coefft':coef_nodes_all_lme_run[GLM_trait_nodes_10resp],
'pval':pval_nodes_all_lme_run[GLM_trait_nodes_10resp], 
'ROI name':[eval(shen268_lbl[str(n+1)][0])['name'] for n in GLM_trait_nodes_10resp]}).sort_values(by=['reg coefft'],ascending = True)

In [ ]:
nrows = 2 #5
ncols= 8
fig, axs = plt.subplots(nrows = nrows, ncols= ncols, figsize=(25,20*.4))#(25,20))
#for ind, n in enumerate(GLM_trait_nodes):    # all subs
for ind, n in enumerate(GLM_trait_nodes_10resp):    # only subs with all 10 responses
    row = int(np.floor(ind/ncols))
    col = int(ind%ncols)
    x = all_coefs_run[:,n,:].flatten()
    x = x[~np.isnan(x)]
    axs[row,col].boxplot(x, widths = 1)
    xmin,xmax = 0,2
    plt.xlim(xmin,xmax)
    axs[row,col].hlines(0,xmin,xmax, color = 'grey', ls = ':')
    plt.ylim(-4,4)
    stat_results =  stats.ttest_1samp(x,0)
    if stat_results[1] < .001:
        suff = '***'
    elif stat_results[1] < .01:
        suff = '**'
    elif stat_results[1] < .05:
        suff = '*'
    elif stat_results[1] < .1:
        suff = '+'
    else:
        suff = ''

    if  stat_results[1] < .05:
        if np.nanmean(x) > 0:
            axs[row,col].set_title(f'N_ind {n},\nM={np.nanmean(x):.2f}{suff}', color = 'red')
        elif np.nanmean(x) < 0:
            axs[row,col].set_title(f'N_ind {n},\nM={np.nanmean(x):.2f}{suff}', color = 'blue')
    else:
        axs[row,col].set_title(f'N_ind {n},\nM={np.nanmean(x):.2f}{suff}')


if ncols > 1:
     #for ind in range(len(GLM_trait_nodes), int(nrows*ncols)):
     for ind in range(len(GLM_trait_nodes_10resp), int(nrows*ncols)):
        row = int(np.floor(ind/ncols))
        col = int(ind%ncols)
        axs[row,col].axis('off')
          
plt.tight_layout()

In [ ]:
nrows = 5
ncols= 8
fig, axs = plt.subplots(nrows = nrows, ncols= ncols, figsize=(25,20))
#for ind, n in enumerate(NoGLM_trait_nodes):
for ind, n in enumerate(NoGLM_trait_nodes_10resp):    # only subs with all 10 responses
    row = int(np.floor(ind/ncols))
    col = int(ind%ncols)
    x = all_coefs_run[:,n,:].flatten()
    x = x[~np.isnan(x)]
    axs[row,col].boxplot(x, widths = 1)
    xmin,xmax = 0,2
    plt.xlim(xmin,xmax)
    axs[row,col].hlines(0,xmin,xmax, color = 'grey', ls = ':')
    plt.ylim(-4,4)
    stat_results =  stats.ttest_1samp(x,0)
    if stat_results[1] < .001:
        suff = '***'
    elif stat_results[1] < .01:
        suff = '**'
    elif stat_results[1] < .05:
        suff = '*'
    elif stat_results[1] < .1:
        suff = '+'
    else:
        suff = ''

    if  stat_results[1] < .05:
        if np.nanmean(x) > 0:
            axs[row,col].set_title(f'N_ind {n},\nM={np.nanmean(x):.2f}{suff}', color = 'red')
        elif np.nanmean(x) < 0:
            axs[row,col].set_title(f'N_ind {n},\nM={np.nanmean(x):.2f}{suff}', color = 'blue')
    else:
        axs[row,col].set_title(f'N_ind {n},\nM={np.nanmean(x):.2f}{suff}')

if ncols > 1:
     for ind in range(len(NoGLM_trait_nodes_10resp), int(nrows*ncols)):
        row = int(np.floor(ind/ncols))
        col = int(ind%ncols)
        axs[row,col].axis('off')
          
plt.tight_layout()

In [ ]:
# Uncorrected overla
x = np.where((coef_nodes_all_lme_run>0) & (pval_nodes_all_lme_run_unc))[0]
print('Unc:\n', [[n,eval(shen268_lbl[str(n+1)][0])['name']] for n in x])

x =  np.where((coef_nodes_all_lme_run>0) & (pval_nodes_fdr_all_lme_run))[0] #np.where(coef_nodes_all_lme_run[pval_nodes_fdr_all_lme_run]>0)[0]
print('\nFDR-corr:\n', [[n,eval(shen268_lbl[str(n+1)][0])['name']] for n in x])

x = np.where((coef_nodes_all_lme_run_10resp>0) & (pval_nodes_all_lme_run_unc_10resp))[0]
print('\nUnc:\n', [[n,eval(shen268_lbl[str(n+1)][0])['name']] for n in x])

x =  np.where((coef_nodes_all_lme_run_10resp>0) & (pval_nodes_fdr_all_lme_run_10resp))[0] #np.where(coef_nodes_all_lme_run[pval_nodes_fdr_all_lme_run]>0)[0]
print('\nFDR-corr:\n', [[n,eval(shen268_lbl[str(n+1)][0])['name']] for n in x])

In [ ]:
n = 121
data = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
for m in range(10):
    fMRI_data = pd.DataFrame({'Subject':np.array(sub_id_all)[subs_10resp],'slopeReg_node': all_coefs_run[subs_10resp,n,m], 
    'response': responses[subs_10resp,m],'nresp':np.array(nresp)[subs_10resp], 'movie': np.repeat(m,subs_10resp.shape[0])})
    fMRI_data.set_index("Subject", inplace=True)
    data_temp = fMRI_data.join(behav_data['ASR_Intn_T'], how='inner') # join betas and trait info
    data = data.append(data_temp)#,ignore_index=True)
data.reset_index(inplace=True)     
inds = ~np.isnan(data['slopeReg_node']) & ~np.isnan(data['ASR_Intn_T']) # find rows where neither x or y is NaN
data = data.loc[inds,:]

avg, sem = data[['slopeReg_node','ASR_Intn_T','Subject']].groupby(['Subject']).mean(), data[['slopeReg_node','ASR_Intn_T','Subject']].groupby(['Subject']).sem()
explore_correlation(x='ASR_Intn_T',y='slopeReg_node', data = avg)

model2 = Lmer('slopeReg_node ~  ASR_Intn_T + (1|movie)', data=data) # ff: mean response, rf:subjID
model2.fit()


In [ ]:
def plot_result_brainmaps(nodes,  contours_mat, intersection_, params,text_x,suff):
    cmap = 'RdBu_r'

    #levels,labels,colors,coods = params['levels'],params['labels'],params['colors'],params['coords']
    colors, coods, vmin, vmax, txt , title_txt = \
        params['color'], params['coords'], params['vmin'], params['vmax'], params['txt'], params['title_txt']

    img = roi_to_brain(pd.Series(nodes), mask_x)
    ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords = coords,cmap = cmap, bg_img = bg_img,
    colorbar= False,threshold = .0001)
    ax_plot.add_contours(color_rois((intersection_)),linewidths=.5, colors=[colors[0]],linestyles='solid') # GLM nodes
    plt.savefig(os.path.join(fig_save_loc,f'ALL/axial{suff}.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

    fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
    
    #LH (left hemisphere)
    ax1 = ax[0,0] # left lateral
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left,axes=ax1)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
     
    texture_contour = vol_to_surf(color_rois(contours_mat), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_left, texture_contour, axes = ax1, figure=surf_plot1, legend=False, levels = [1], colors=colors)

    ax1 = ax[1,0] # left medial
    surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin,\
         vmax = vmax, bg_map=fsaverage.sulc_left, view = 'medial',axes=ax1)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_left, texture_contour, axes = ax1, figure=surf_plot2, legend=True, levels = [1],colors=colors)

    #RH (right hemisphere)
    ax1 = ax[0,1] # right lateral
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right,axes=ax1)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    texture_contour = vol_to_surf(color_rois(contours_mat), fsaverage.pial_right,interpolation='nearest',radius = 1, n_samples=1)
    plot_surf_contours(fsaverage.infl_right, texture_contour, axes = ax1, figure=surf_plot3, legend=False,levels = [1],colors=colors)
    
    box = surf_plot3.axes[4].get_position()
    surf_plot3.axes[4].set_position([box.x0*.93, box.y0-.3, box.width, box.height*2])  # move a bit the bar to the right, need to divide by number of columns (to move relative to last figure only, not to overall row, else will get too far away)
    
    ax1 = ax[1,1] # right medial
    surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right, view ='medial',axes=ax1)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_right, texture_contour, axes = ax1, figure=surf_plot4, levels = [1], colors=colors)
    
    ax[0,0].dist = 7 # change viewing distance to "zoom in" to surface plots
    ax[0,1].dist = 7
    ax[1,0].dist = 7
    ax[1,1].dist = 7
    
    plt.subplots_adjust(left=0,
                        bottom=0, 
                        right=.8, 
                        top=1, 
                        wspace=0.0, 
                        hspace=-.1)
    plt.savefig(os.path.join(fig_save_loc,f'ALL/surf_all4{suff}.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    

In [ ]:
# only subs with all 10 responses
coords = [-44,-34,-24] # for the final version
cmap = 'RdBu_r'
   
vmin,vmax = -.006,.006
txt = 'LME\nEstimate'

nodes = np.zeros((268,))
nodes[pval_nodes_fdr_all_lme_run_10resp] = coef_nodes_all_lme_run_10resp[pval_nodes_fdr_all_lme_run_10resp]

title_txt = "" #title_txt = r"    Trait-fMRI correlation, all animations" # (nodes unc.)'

contours_mat = np.zeros((268,))
contours_mat[nodes_coaxbill_rand_all] = 1

params = {'color': ['k'], 'coords': coords, 'vmin':vmin, 'vmax': vmax, 'txt': txt , 'title_txt': title_txt}
text_x = 7
suff = '_all_only10subs'
plot_result_brainmaps(nodes, contours_mat, nodes_coaxbill_rand_all, params, text_x,suff)
